### Imports

In [2]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import get_dataset_class
# from plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

DS = "TerraIncognita"
MODEL_NAME = "WinKawaks/vit-small-patch16-224"

Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [3]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

DatasetClass = get_dataset_class(DS)

ld = DatasetClass(
    data_root=CFG["datasets"][DS]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [4]:
dataset_name = DS
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-20 13:10:08,417 | INFO | Initialized experiment directories for TerraIncognita
2025-10-20 13:10:08,417 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\TerraIncognita\logs
2025-10-20 13:10:08,417 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\TerraIncognita\checkpoints
2025-10-20 13:10:08,417 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\TerraIncognita\plots


### Setup

In [5]:
domains = CFG["datasets"][DS]["domains"]
loaders = {d: {"train": ld.get_dataloader(d, train=True), "val": ld.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset=DS)
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]


{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [6]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-20 13:10:12,049 | INFO | === LODO: Leaving out domain 'location_38' ===



=== LODO: Leaving out domain 'location_38' ===


Evaluating: 100%|██████████| 77/77 [00:59<00:00,  1.30it/s]
2025-10-20 13:13:01,287 | INFO | [location_38] Epoch 1/5 | Train - Loss: 1.0538, Cls: 1.0483, GRQO: 0.0055, Acc: 0.6296 | Val - Loss: 1.9390, Cls: 1.9382, GRQO: 0.0008, Acc: 0.4156
2025-10-20 13:13:01,384 | INFO | [location_38] New best val acc: 0.4156


[location_38] Epoch 1/5 | Train - Loss: 1.0538, Cls: 1.0483, GRQO: 0.0055, Acc: 0.6296 | Val - Loss: 1.9390, Cls: 1.9382, GRQO: 0.0008, Acc: 0.4156
[location_38] New best val acc: 0.4156


Evaluating: 100%|██████████| 77/77 [00:56<00:00,  1.36it/s]
2025-10-20 13:15:35,231 | INFO | [location_38] Epoch 2/5 | Train - Loss: 0.4947, Cls: 0.4923, GRQO: 0.0023, Acc: 0.8346 | Val - Loss: 2.7346, Cls: 2.7342, GRQO: 0.0005, Acc: 0.3147


...

[location_100] New best val acc: 0.5819


Evaluating: 100%|██████████| 39/39 [00:28<00:00,  1.38it/s]
2025-10-20 14:09:36,458 | INFO | [location_100] Epoch 4/5 | Train - Loss: 0.2088, Cls: 0.2073, GRQO: 0.0015, Acc: 0.9290 | Val - Loss: 2.0772, Cls: 2.0768, GRQO: 0.0003, Acc: 0.5038


[location_100] Epoch 4/5 | Train - Loss: 0.2088, Cls: 0.2073, GRQO: 0.0015, Acc: 0.9290 | Val - Loss: 2.0772, Cls: 2.0768, GRQO: 0.0003, Acc: 0.5038


Evaluating: 100%|██████████| 39/39 [00:28<00:00,  1.37it/s]
2025-10-20 14:12:16,573 | INFO | [location_100] Epoch 5/5 | Train - Loss: 0.1594, Cls: 0.1581, GRQO: 0.0012, Acc: 0.9460 | Val - Loss: 2.3089, Cls: 2.3086, GRQO: 0.0003, Acc: 0.5042
2025-10-20 14:12:16,574 | INFO | [location_100] Best Acc: 0.5819
2025-10-20 14:12:16,574 | INFO | ------------------------------------------------------------
2025-10-20 14:12:16,575 | INFO | LODO finished | Mean Acc: 0.4767 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\TerraIncognita\logs\lodo_summary_20251020_141216.json


[location_100] Epoch 5/5 | Train - Loss: 0.1594, Cls: 0.1581, GRQO: 0.0012, Acc: 0.9460 | Val - Loss: 2.3089, Cls: 2.3086, GRQO: 0.0003, Acc: 0.5042
[location_100] Best Acc: 0.5819
------------------------------------------------------------
LODO finished | Mean Acc: 0.4767
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-small\TerraIncognita\logs\lodo_summary_20251020_141216.json


### Baseline

In [7]:
baseline_results, baseline_mean = run_baseline(
    model_name=MODEL_NAME,
    CFG=CFG,
    logger=logger,
    dataset_key=DS,
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)


2025-10-20 14:12:16,593 | INFO | Initializing ViT baseline: WinKawaks/vit-small-patch16-224


Initializing ViT baseline: WinKawaks/vit-small-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([10, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-20 14:12:17,623 | INFO | === Baseline LODO: Leaving out domain 'location_38' ===



=== Baseline LODO: Leaving out domain 'location_38' ===


2025-10-20 14:14:48,600 | INFO | [location_38] Epoch 1/5 | Train - Loss: 0.8359, Acc: 0.7085 | Val Acc: 0.3450


...

[location_100] Epoch 3/5 | Train - Loss: 0.1956, Acc: 0.9341 | Val Acc: 0.5725


2025-10-20 15:03:35,073 | INFO | [location_100] Epoch 4/5 | Train - Loss: 0.1426, Acc: 0.9516 | Val Acc: 0.5809


[location_100] Epoch 4/5 | Train - Loss: 0.1426, Acc: 0.9516 | Val Acc: 0.5809


2025-10-20 15:06:11,865 | INFO | [location_100] Epoch 5/5 | Train - Loss: 0.1188, Acc: 0.9593 | Val Acc: 0.4778
2025-10-20 15:06:11,865 | INFO | [location_100] Best Val Acc: 0.5977
2025-10-20 15:06:11,865 | INFO | ------------------------------------------------------------
2025-10-20 15:06:11,865 | INFO | Baseline LODO (WinKawaks/vit-small-patch16-224) finished | Mean Acc: 0.4700


[location_100] Epoch 5/5 | Train - Loss: 0.1188, Acc: 0.9593 | Val Acc: 0.4778
[location_100] Best Val Acc: 0.5977
------------------------------------------------------------
Baseline LODO (WinKawaks/vit-small-patch16-224) finished | Mean Acc: 0.4700
